In [4]:
import pandas as pd
import numpy as np
import os
from pandas import Series, DataFrame
group_train = pd.read_csv("C:\\Users\\karis\\NewGroup_data.txt", sep="\t", header = None, names= ['phrase_id', 'review', 'group_id'])
group_train.head()

,phrase_id,review,group_id
0,1,"['From: mathew <mathew@mantis.co.uk>\n', 'Subj...",0
1,2,"['From: mathew <mathew@mantis.co.uk>\n', 'Subj...",0
2,3,['From: I3150101@dbstu1.rz.tu-bs.de (Benedikt ...,0
3,4,"['From: mathew <mathew@mantis.co.uk>\n', 'Subj...",0
4,5,"['From: strom@Watson.Ibm.Com (Rob Strom)\n', '...",0


In [2]:
group_test = pd.read_csv("C:\\Users\\karis\\NewGroup_data_test.txt", sep="\t", header = None, names= ['phrase_id', 'review', 'group_id'])
group_test.head()

,phrase_id,review,group_id
0,1,['From: decay@cbnewsj.cb.att.com (dean.kaflowi...,0
1,2,"['From: cfaehl@vesta.unm.edu (Chris Faehl)\n',...",0
2,3,"['From: mathew <mathew@mantis.co.uk>\n', 'Subj...",0
3,4,"['From: dps@nasa.kodak.com (Dan Schaertel,,,)\...",0
4,5,"['From: halat@panther.bears (Jim Halat)\n', 'S...",0


In [3]:
# maping cateogories to train dataframe

group_train['category'] = group_train.group_id.map({0:'alt', 1:'comp',2:'comp', 3:'comp', 4:'comp',5:'comp',6:'misc', 7:'rec', 8:'rec',9:'rec', 10:'rec', 11:'sci',
                                    12:'sci', 13:'sci', 14:'sci', 15:'religion', 
                                    16:'politics', 17:'politics', 18:'politics', 19:'religion'})

# maping cateogories to test dataframe

group_test['category'] = group_test.group_id.map({0:'alt', 1:'comp',2:'comp', 3:'comp', 4:'comp',5:'comp',6:'misc', 7:'rec', 8:'rec',9:'rec', 10:'rec', 11:'sci',
                                    12:'sci', 13:'sci', 14:'sci', 15:'religion', 
                                    16:'politics', 17:'politics', 18:'politics', 19:'religion'})

group_test.shape

(7532, 4)

In [4]:
# selecting categories 
group_train_category = group_train[(group_train.category == 'comp') | (group_train.category == 'rec') | (group_train.category == 'politics') | (group_train.category == 'religion')]
group_test_category = group_test[(group_test.category == 'comp') | (group_test.category == 'rec') | (group_test.category == 'politics') | (group_test.category == 'religion')]



# # spliting data into input X_train and output Y_train
# X_train = group_train_category.review
# # Y_train = group_train_category.category

# # spliting data into input X_train and output Y_train
# X_test = group_test_category.review
# # Y_test = group_test_category.category

# print(X_train.shape)
# print(X_test.shape)
# group_train_category.head()

In [5]:
group_train_category['category_num'] = group_train_category.category.map({'comp':0, 'rec':1, 'politics':2, 'religion':3})
group_test_category['category_num'] = group_test_category.category.map({'comp':0, 'rec':1, 'politics':2, 'religion':3})

group_train_test_category = pd.concat([group_train_category, group_test_category])

C:\Users\karis\Anaconda2\envs\py3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\karis\Anaconda2\envs\py3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [6]:
print(group_train_category.shape)
print(group_test_category.shape)
print(group_train_test_category.shape)

(7876, 5)
(5244, 5)
(13120, 5)


In [7]:
import numpy as np
from keras.utils import np_utils

Y = np_utils.to_categorical(group_train_test_category['category_num'])
X = group_train_test_category.review

# Y_test_label_encoded = np_utils.to_categorical(np.asarray(group_test_category['category']))


print(Y.shape)
print(X.shape)



# from sklearn.preprocessing import LabelEncoder


# Label_encoded = LabelEncoder()
# Y_train_label_encoded = Label_encoded.fit_transform(group_train_category['category'].values)
# Y_test_label_encoded = Label_encoded.fit_transform(group_test_category['category'].values)



Using TensorFlow backend.


(13120, 4)
(13120,)


In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X)

X_sequences = tokenizer.texts_to_sequences(X)

X_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(X_word_index))

X_data = pad_sequences(X_sequences, maxlen=1000)

print('Shape of data tensor:', X_data.shape)



Found 172367 unique tokens.
Shape of data tensor: (13120, 1000)


In [9]:
import os
import numpy as np
embeddings_index = {}
f = os.path.join('C:/Users/karis/Desktop/latently', 'glove.6B.100d.txt')
with open(f,encoding='utf-8') as main:
    for line in main:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs


print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [10]:
embedding_matrix = np.zeros((len(X_word_index) + 1, 100))
for word, i in X_word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


In [80]:
embedding_matrix.shape

(172368, 100)

In [44]:
from keras.layers import Embedding

embedding_layer = Embedding(len(X_word_index) + 1,
                            100,
                            weights=[embedding_matrix],
                            input_length=1000,
                            trainable=False)

In [11]:
X_train = X_data[:10000]  
X_test = X_data[10000:] 
Y_train = Y[:10000]
Y_test = Y[10000:]

print(Y_train.shape)
print(X_train.shape)
print(Y_test.shape)
print(X_test.shape)

(10000, 4)
(10000, 1000)
(3120, 4)
(3120, 1000)


In [12]:
import numpy
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Convolution1D
from keras.layers import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.models import Model

In [15]:
seed = 7
numpy.random.seed(seed)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
max_words = 500

# create the model
model = Sequential()
model.add(Embedding(len(X_word_index) + 1, 100, weights=[embedding_matrix], input_length=1000, trainable=False)
model.add(Flatten())
model.add(Dense(250, activation= 'relu' ))
model.add(Dense(1, activation= 'sigmoid' ))
model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
print(model.summary())
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=2, batch_size=128,
verbose=1)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

SyntaxError: invalid syntax (<ipython-input-15-eead0413fb81>, line 11)

In [74]:
# sequence_input = Input(shape=(1000,), dtype='int32')
# embedded_sequences = embedding_layer(sequence_input)
# x = Conv1D(128, 5, activation='relu')(embedded_sequences)
# x = MaxPooling1D(5)(x)
# x = Conv1D(128, 5, activation='relu')(x)
# x = MaxPooling1D(5)(x)
# x = Conv1D(128, 5, activation='relu')(x)
# x = MaxPooling1D(35)(x)  # global max pooling
# x = Flatten()(x)
# x = Dense(128, activation='relu')(x)
# preds = Dense(len(Y_train), activation='softmax')(x)

# model = Model(sequence_input, preds)
# model.compile(loss='categorical_crossentropy',
#               optimizer='rmsprop',
#               metrics=['acc'])

# # happy learning!
# model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=2, batch_size=100)